# Lecture 5 –  Accessing, Sorting, and Querying
## DSC 10, Winter 2022

### Announcements

- Homework 1 is due on **Saturday 1/15 at 11:59pm**.
    - Hint for Question 9: The question we want to try and answer with an RCT is "Does winning the lottery increase one's chance of going bankrupt?", not "Does receiving a lot of money increase one's chance of going bankrupt?" There is a subtle but important difference between these two questions!
- Lab 2 is due on **Tuesday 1/18 at 11:59pm**.
- Remember to check the [Debugging](https://dsc10.com/debugging) tab on the course website if you run into any issues.

### Agenda

- Using a single dataset to illustrate key DataFrame manipulation techniques.
    - Will use lots of motivating questions.

#### Note:

- Remember to check the [Resources tab of the course website](https://dsc10.com/resources/) for programming resources.
- Some key links moving forward:
    - [DSC 10 Reference Sheet](https://drive.google.com/file/d/1mQApk9Ovdi-QVqMgnNcq5dZcWucUKoG-/view).
    - [BabyPandas Documentation](https://babypandas.readthedocs.io/en/latest/index.html).

### Extra video from Lecture 4

Make sure to watch this video for context.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('t_bjtBJ0gGc')

## DataFrames

### `pandas`

- `pandas` is **the** tool for doing data science in Python, but it can be hard to learn as a beginner.

<center>
<img height=75% src="data/angrypanda.jpg"/ width=500>
</center>

### `babypandas`

- We at UCSD have created a smaller, nicer version of `pandas` called `babypandas`.
- It keeps the important stuff and throws out the rest.
- It's easier to learn, but is still valid `pandas` code.

<center>
<img height=75% src="data/babypanda.jpg"/ width=500>
</center>

### DataFrames in `babypandas` 🐼

- Tables in `babypandas` (and `pandas`) are called "DataFrames."
- To use DataFrames, we'll need to import `babypandas`. (We'll need `numpy` as well.)

In [ ]:
import babypandas as bpd
import numpy as np

### Reading data from a file

- The file `nba-2022.csv` contains all salaries from the 2021-22 NBA (National Basketball Association 🏀) season for players who have played at least 15 games.
    - CSV stands for "comma-separated values".
- We can read a CSV using `bpd.read_csv(...)`. Give it the name of the file if it's in the same directory as your notebook, or a path to the file otherwise.

In [ ]:
salaries = bpd.read_csv('data/nba-2022.csv')
salaries

### Structure of a DataFrame

- DataFrames have *columns* and *rows*.
    - Think of each column as an array.
- Each column has a label: `'Player'`, `'Position'`, etc.
    - This is its name.
    - Column labels are stored as strings.
- Every row has a label too: in this case, 0, 1, 2, 3, 4, ..., 380.
    - Together, the row labels are called the _index_. The index is **not** a column!

In [ ]:
salaries

### Setting a new index

- We can set a better index using `.set_index(column_name)`.
- Row labels should be unique identifiers.
    - Row labels = row names; ideally, each row has a different, descriptive name.
- Like most **DataFrame methods**, `.set_index` returns a new DataFrame; it does not modify the original DataFrame.
- The result not only looks nicer, but will be easier to manipulate, as we'll see soon.

In [ ]:
salaries_by_player = salaries.set_index('Player')
salaries_by_player

### Shape of a DataFrame

- `.shape` returns the number of rows and columns in a given DataFrame.
- Access each with `[]`:

In [ ]:
salaries_by_player.shape

In [ ]:
# Number of rows
salaries_by_player.shape[0]

In [ ]:
# Number of columns
salaries_by_player.shape[1]

## Example 1: Salary per game

**Key concepts:** How do we access columns, perform operations on them, and add new columns?

### Finding per-game salaries

- We have each player's salary this season. An NBA regular season has 82 games in it.
- **Question:** How much money is each player getting paid per game?
- Workflow:
    - Get the column of salaries.
    - Divide each element by 82.
    - Add a new column to the DataFrame.

#### Step 1 – Getting a column

- We can get a column from a DataFrame using `.get(column_name)`.
- Warning: column names are case sensitive!
- The result looks like a 1-column DataFrame, but is actually a *Series*.

In [ ]:
salaries_by_player

In [ ]:
salaries_by_player.get('Salary')

### Digression: Series

- A *Series* is like an array, but with an index.
- In particular, Series' support arithmetic.

In [ ]:
salaries_by_player.get('Salary')

#### Step 2 – Dividing salaries by the number of games

- Just like with arrays, we can perform arithmetic operations on every element of a Series quite efficiently.

In [ ]:
salaries_by_player.get('Salary') / 82

#### Step 3 – Adding the per-game salaries to the DataFrame

- Use `.assign(Name_of_column=data_in_array)` to assign a Series (or array, or list) to a DataFrame.
- **Warning!** Don't put quotes around `Name_of_column`.
- Creates a new DataFrame! Must save to variable.

In [ ]:
salaries_by_player.assign(
    Salary_per_game=salaries_by_player.get('Salary') / 82
)

In [ ]:
salaries_by_player

In [ ]:
# pg stands for "per game"
salaries_pg = salaries_by_player.assign(
    Salary_per_game=salaries_by_player.get('Salary') / 82
)

In [ ]:
salaries_pg

## Example 2: Getting a particular player's salary

**Key concept**: How do we access specific values using an index?

### How much is LeBron James 🐐 making this season?

In [ ]:
# This is a Series!
salaries_pg.get('Salary')

### Accessing a Series by row label: `.loc`

- Use `.loc[]` to *access* the element of a Series with a particular row label.

In [ ]:
salaries_pg.get('Salary').loc['LeBron James']

### How to get a particular element from a DataFrame

1. `.get()` the column label.
2. `.loc[]` the row label.

In this class, we'll always get a column, then a row (but row, then column is also possible).

 Example: What position does LeBron play?

In [ ]:
salaries_pg.get('Position').loc['LeBron James']

## Example 3: Salary analysis

**Key concept**: How do we compute statistics of columns?

### Questions about salary

- What is the highest/lowest salary? What is the average salary?
- Series' have helpful methods, like `.min()`, `.max()`, `.mean()`, etc.

In [ ]:
salaries_pg.get('Salary').min()

In [ ]:
salaries_pg.get('Salary').max()

In [ ]:
salaries_pg.get('Salary').mean()

In [ ]:
salaries_pg.get('Salary').median()

## Example 4: *Who* has the highest salary?

**Key concept**: Sorting.

#### Step 1 – Sorting the DataFrame

- Use the `.sort_values(by=column_name)` method to sort.
    - **Notice:** Like most DataFrame methods, this returns a new DataFrame.
- Everything works as expected, but we wanted *descending* order.

In [ ]:
salaries_pg.sort_values(by='Salary')

#### Step 1 – Sorting the DataFrame in *descending* order

- Use `.sort_values(by=column_name, ascending=False)` to sort in *descending* order.

In [ ]:
highest_salaries = salaries_pg.sort_values(by='Salary', ascending=False)
highest_salaries

#### Step 2 – Getting the *name* of the person with the highest salary

- We saw that it's Stephen Curry, but how do we get the name using code?
- Remember, the index is like an array.

In [ ]:
highest_salaries.index[0]

## Example 5: What team does the person with the third-lowest salary play for?

**Key concept**: Using integer positions.

#### Strategy 1

Using what we already know, we could...

1. Sort the DataFrame in ascending order using `.sort_values(by='Salary')`.
2. Get the name of the person using `.index[2]` (remember, positions start at 0).
3. Use `.get('Team').loc[their_name]` to get their team name.



In [ ]:
lowest_salaries = salaries_pg.sort_values(by='Salary')
lowest_salaries

In [ ]:
name = lowest_salaries.index[2]
name

In [ ]:
lowest_salaries.get('Team').loc[name]

#### Another approach?

- To get the third element of a Series using `.loc[]`, we first had to find its label.
- Can we just get the third element of a Series without knowing its label?
- Yes, with `.iloc[]`:

In [ ]:
lowest_salaries.get('Team')

In [ ]:
lowest_salaries.get('Team').loc['Alize Johnson']

In [ ]:
lowest_salaries.get('Team').iloc[2]

#### Strategy 2

1. Sort the DataFrame in ascending order using `.sort_values(by='Salary')`, as before.
2. Use `.get('Team').iloc[2]` to get the team name.

In [ ]:
salaries_pg.sort_values(by='Salary').get('Team').iloc[2]

### Summary of accessing a Series

- There are two ways to get an element of a Series:
    - `.loc[]` uses the row label.
    - `.iloc[]` uses the integer position.
- Usually `.loc[]` is more convenient, but you'll need to know both.

### Note

- Sometimes the integer position and row label are the same.
- This happens by default with `bpd.read_csv`:

In [ ]:
bpd.read_csv('data/nba-2022.csv')

In [ ]:
bpd.read_csv('data/nba-2022.csv').get('Player').loc[3]

In [ ]:
bpd.read_csv('data/nba-2022.csv').get('Player').iloc[3]

## Reflection

### Questions we can answer right now:
- What is the highest salary?
    - `salaries_pg.get('Salary').max()`
- How many players are there in the dataset?
    - `salaries_pg.shape[0]`
- What is LeBron James' salary per game?
    - `salaries_pg.get('Salary_per_game').loc['LeBron James']`
- Who has the highest salary?
    - `salaries_pg.sort_values(by='Salary', ascending=False).index[0]`

### Questions we can't yet answer:
- What is the total payroll of the Cleveland Cavaliers?
- How many players make over \$200,000 per game?
- Who is the highest paid center (C)?

The common thread between these questions is that they all involve only a **subset** of the rows in our DataFrame.

## Example 6: Who is the highest paid center (C)?

**Key concept**: Selecting rows (via Boolean indexing).

### Selecting rows

- We can determine who the highest paid center (C) is if we have a DataFrame consisting of only centers.
- How do we get that DataFrame?

### The solution

In [ ]:
salaries_pg[salaries_pg.get('Position') == 'C']

In [ ]:
'PG' == 'C'

In [ ]:
'C' == 'C'

In [ ]:
salaries_pg.get('Position') == 'C'

### Boolean indexing

To select only some rows of `salaries_pg`:

1. Make a sequence (list/array/Series) of `True`s (keep) and `False`s (toss).
    - The values `True` and `False` are of the _Boolean_ data type.
2. Then pass it into `salaries_pg[sequence_goes_here]`.

Rather than making the sequence by hand, we usually generate it by making a comparison.

### Elementwise comparisions work as expected

In [ ]:
salaries_pg.get('Salary_per_game') > 200000

### How do we make a DataFrame with only the players who make over \$200,000 per game?

In [ ]:
salaries_pg[salaries_pg.get('Salary_per_game') > 200000]

### How do we make a DataFrame with only the players on the Cleveland Cavaliers?

In [ ]:
salaries_pg[salaries_pg.get('Team') == 'Cleveland Cavaliers']

### Original Question: How do we determine who the highest paid center is?

Strategy:
1. Extract a DataFrame of just the centers.
2. Sort by salary.
3. Return the first element in the index.

In [ ]:
centers = salaries_pg[salaries_pg.get('Position') == 'C']
highest_paid_center = centers.sort_values(by='Salary', ascending=False).index[0]
highest_paid_center

### What if the condition isn't satisfied?

In [ ]:
salaries_pg[salaries_pg.get('Team') == 'UC San Diego Tritons']

### Discussion Question

Which of these three queries determines the total payroll of the Los Angeles Lakers?

A. `salaries_pg[salaries_pg.get('Team') == 'Los Angeles Lakers'].get('Salary').sum()`

B. `salaries_pg.get('Salary').sum()[salaries_pg.get('Team') == 'Los Angeles Lakers']`

C. `salaries_pg['Los Angeles Lakers'].get('Salary').sum()`

### To answer, go to **[menti.com](https://menti.com)** and enter the code **5313 8252**.

## Summary

### Summary

- We learned many DataFrame methods and techniques.
- Don't feel the need to memorize them all right away.
- Instead, use this lecture and the aforementioned readings as references when working on the labs and homeworks.
- Over time, these techniques will become more and more familiar.
- **Practice!** Frame your own questions using this dataset and try and answer them.

### Next time

- On Friday, we'll try and answer more involved questions involving our data, which will lead us to a new core DataFrame method – `groupby`.